In [1]:
import os
import json
import re
import pandas as pd

# Pfad zum Hauptverzeichnis mit den Unterverzeichnissen
BASE_DIR = "data/liboqs"

def extract_model_name(directory_name):
    """Extrahiert den Modelnamen aus dem Verzeichnisnamen."""
    match = re.match(r"testresult_liboqs_(.*?)_\d{4}\.\d{2}\.\d{2}_\d{2}\.\d{2}", directory_name)
    return match.group(1) if match else directory_name

# Lade die JSON-Datei
def load_kem_memory_data(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

# Extrahiere relevante Daten aus der JSON-Datei
def parse_kem_memory_data(data, model_name):
    records = []
    for algorithm, details in data.items():
        if algorithm in ["cpuinfo", "config"]:
            continue  # Überspringe Meta-Daten

        keygen = details.get("keygen", {})
        encaps = details.get("encaps", {})
        decaps = details.get("decaps", {})

        records.append({
            "Model": model_name,
            "Algorithmus": algorithm,
            "maxHeap_key": keygen.get("maxHeap", 0),
            "maxHeap_enc": encaps.get("maxHeap", 0),
            "maxHeap_dec": decaps.get("maxHeap", 0),
            "maxStack_key": keygen.get("maxStack", 0),
            "maxStack_enc": encaps.get("maxStack", 0),
            "maxStack_dec": decaps.get("maxStack", 0)
        })
    
    return records

# Hauptfunktion zum Laden und Verarbeiten der Daten
def main():
    all_records = []
    
    for directory in os.listdir(BASE_DIR):
        full_path = os.path.join(BASE_DIR, directory)
        if os.path.isdir(full_path):
            json_file = os.path.join(full_path, "test_kem_mem_fullHeap.json")
            if os.path.isfile(json_file):
                model_name = extract_model_name(directory)
                data = load_kem_memory_data(json_file)
                recs = parse_kem_memory_data(data, model_name)
                all_records.extend(recs)
    
    # DataFrame erstellen
    df = pd.DataFrame(all_records)
    df = df.sort_values(by=["Algorithmus", "Model"])
    
    # Speichere als CSV
    df.to_csv("output/csv/liboqs_mem_kem_fullHeap.csv", index=False)
    
    # Anzeige im Jupyter Notebook
    from IPython.display import display
    display(df)
    
    return df

# Starte die Verarbeitung
df = main()


,Model,Algorithmus,maxHeap_key,maxHeap_enc,maxHeap_dec,maxStack_key,maxStack_enc,maxStack_dec
52,Pi3b,BIKE-L1,6135808,6135808,6135808,0,0,0
81,Pi4b,BIKE-L1,6135808,6135808,6135808,0,0,0
136,PiZero2,BIKE-L1,6094848,6094848,6094848,0,0,0
53,Pi3b,BIKE-L3,6135808,6135808,6135808,0,0,0
82,Pi4b,BIKE-L3,6135808,6135808,6135808,0,0,0
...,...,...,...,...,...,...,...,...
74,Pi3b,sntrup761,6135808,6135808,6135808,0,0,0
103,Pi4b,sntrup761,6135808,6135808,6135808,0,0,0
129,PiBanana,sntrup761,8716288,8716288,8716288,0,0,0
181,PiZero,sntrup761,5136384,5136384,5136384,0,0,0
